In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import numpy as np
import pandas as pd
import ray 

from dr.dr import apply_dr

In [11]:
ray.shutdown()
ray.init(memory=10e+9, object_store_memory=5e+9)


2020-04-16 19:09:42,522	INFO resource_spec.py:216 -- Starting Ray with 9.28 GiB memory available for workers and up to 4.66 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).


{'node_ip_address': '192.168.103.29',
 'redis_address': '192.168.103.29:56397',
 'object_store_address': '/tmp/ray/session_2020-04-16_19-09-42_502157_25499/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-04-16_19-09-42_502157_25499/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2020-04-16_19-09-42_502157_25499'}

In [8]:
f = 'paintings'
size=200
c = 10

input_file = 'data/{}/flatfiles/{}_flat{}_{}classes.csv'.format(f,f,size,c)
output_folder = 'data/{}/dr_data/'.format(f)
dr_name = 'PCA'



dr_data = apply_dr(input_file =input_file,
                  output_folder=output_folder, 
                  dataset_name = f, 
                  dr_name=dr_name)

---------Startings: paintings - paintings_PCA-----------
---------Finished: paintingspaintings_PCA-----------


In [7]:
# apply all these drs

files = ['coil-100','caltech','paris_buildings', 'pets', 'oxford_buildings','dogs', 'flowers','paintings']
files = ['paintings']


print(files)
size = 200

for f in files:
    input_file = 'data/{}/flatfiles/{}_flat{}.csv'.format(f,f,size)
    output_folder = 'data/{}/dr_data/'.format(f)
    
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for dr_name in ['PCA','TSNE', 'ISM', 'UMAP']:
        print(dr_name)
        
        if dr_name in ['PCA','GRP']:
            dr_data = apply_dr(input_file =input_file,
                                      output_folder=output_folder, 
                                      dataset_name = f, 
                                      dr_name=dr_name)
            
        elif dr_name == 'TSNE':
            for p in [5,10,30,50,150,100]:
                #print(p)
                dr_data = apply_dr.remote(input_file =input_file,
                                      output_folder=output_folder, 
                                      dataset_name = f, 
                                      dr_name=dr_name,
                                      perplexity=p)   
        
        else:
            for n in [2,3,5,7,10,15]:
                #print(n)
                if dr_name == 'UMAP':
                    for d in [0.1, 0.5, 0.8]:
                        dr_data = apply_dr.remote(input_file =input_file,
                                      output_folder=output_folder, 
                                      dataset_name = f, 
                                      dr_name=dr_name, n_neighbors=n, min_dist=d)
                elif dr_name in ['LLE','SE']:
                    if n == 7:
                        dr_data = apply_dr.remote(input_file =input_file,
                                      output_folder=output_folder, 
                                      dataset_name = f,  n_neighbors=n)
                    else: 
                        pass
                    
                else:
                    dr_data = apply_dr.remote(dinput_file =input_file,
                                      output_folder=output_folder, 
                                      dataset_name = f, n_neighbors=n)


['paintings']
PCA


Exception: Remote functions cannot be called directly. Instead of running 'dr.dr.apply_dr()', try 'dr.dr.apply_dr.remote()'.